## 3.2.2 Realistic simulation setup
In this script we will change the paradigm. Instead of only using 4 points in the real world to define a circle in the 2D plane, we will use multiple points(~1024).

### import  libraries

In [1]:
vscode = 1

In [2]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
 
from typing import Sequence
from calib_lib import *
from matplotlib.collections import PatchCollection      # pathcoletion for multiple patches
import matplotlib.cm as cm                              # for colormaps
from matplotlib.patches import Rectangle                # for rectangle representation
from matplotlib.patches import Ellipse                  # for ellipse representation   
from pixelization_lib import *      

DECIMALS = 2            # how many decimal places to use in print


In [4]:
F = 16                                       # focal length( in mm )
gridSize = 1024
subpixel_grid = 64
image_size = np.array([1936,1216])               # sensor size(in mm)
#image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                                 # yaw angle

# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44

radius = 5

## play with the

In [6]:
xrange_5 = np.linspace(-400,400,6)
yrange_5 = np.linspace(-120,120,6)
zrange_5 = ([2000])

world_pts_arr,x_zhang_R,x_zhang_L,P_R,P_L = create_1024_point_chess(xrange_5,yrange_5,zrange_5)

(180, 140, 1024, 3)
(140, 1024, 4)
(180, 140, 1024, 3)
World points:  (180, 1, 143360, 3)
Image points:  (180, 140, 1024, 2)


In [13]:
xrange_5 = np.linspace(-400,400,2)
xrange_5

array([-400.,  400.])

In [14]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) * pixel_width
pixelGridY  = (np.arange(0,image_size[1])) * pixel_width
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

fig = plt.figure()
ax = fig.add_subplot()
ax.plot(pixelGridXX,pixelGridYY,'r')
ax.plot(pixelGridXX.T,pixelGridYY.T,'b')

for i in range(90):
    ax.plot(x_zhang_L[i,:,:,0]*pixel_width,x_zhang_L[i,:,:,1]*pixel_width, color = 'r',ls = "None", marker = "*")

plt.show()

In [15]:
def spot_center_coord(cx0,cy0,pixelGridX,pixelGridY):
    cx0_max = np.where((cx0 < pixelGridX))
    cx0_min = np.where((cx0 > pixelGridX))
    #print(cx0_max[0][0])
    #print(cx0_min[0][-1])
    cy0_max = np.where((cy0 < pixelGridY))
    cy0_min = np.where((cy0 > pixelGridY))
    #print(cy0_max[0][0])
    #print(cy0_min[0][-1])

    return cx0_max[0][0],cx0_min[0][-1],cy0_max[0][0],cy0_min[0][-1]

In [16]:
x0_L = x_zhang_L[:,:,:,0]*pixel_width
y0_L = x_zhang_L[:,:,:,1]*pixel_width
x0_L.shape

(180, 140, 1024)

In [19]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) * pixel_width
pixelGridY  = (np.arange(0,image_size[1])) * pixel_width
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

points_max_min_L =np.zeros((x0_L.shape[0],x0_L.shape[1],x0_L.shape[2],2))

for i in range(x0_L.shape[0]):
    for j in range(x0_L.shape[1]):
        for k in range(x0_L.shape[2]):

            cx0max,cx0min,cy0max,cy0min = spot_center_coord(x0_L[i,j,k],y0_L[i,j,k],pixelGridX,pixelGridY)

            # max and min points in (x,y) coordinates
            cx0min_pxl_L = (cx0min*pixel_width)
            cx0max_pxl_L = (cx0max*pixel_width)
            cy0min_pxl_L = (cy0min*pixel_width)
            cy0max_pxl_L = (cy0max*pixel_width)

            #array max and min points
            points_max_min_L[i,j,k,:] = (([cx0min_pxl_L,cy0min_pxl_L]))


In [ ]:
""" fig = plt.figure()
ax = fig.add_subplot()
ax.plot(pixelGridXX,pixelGridYY,'r')
ax.plot(pixelGridXX.T,pixelGridYY.T,'b') """

x_L_lst = []
y_L_lst = []
for chess_b_i in range(x0_L.shape[0]):
    x_L_pts = []
    y_L_pts = []
    for point in range(x0_L.shape[1]):
        #ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
        max_pt_x = np.max(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        max_pt_y = np.max(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        min_pt_x = np.min(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        min_pt_y = np.min(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        #ax.plot(np.array(points_outter_L[chess_b_i][point])[:,0],np.array(points_outter_L[chess_b_i][point])[:,1], color = 'y',ls = "None", marker = "*")
        #ax.plot(max_pt_x,max_pt_y, color = 'r',ls = "None", marker = "*")
        #ax.plot(max_pt_x,min_pt_y, color = 'r',ls = "None", marker = "*")
        #ax.plot(min_pt_x,min_pt_y, color = 'r',ls = "None", marker = "*")
        #ax.plot(min_pt_x,max_pt_y, color = 'r',ls = "None", marker = "*")

        pts = np.array([[max_pt_x,max_pt_y],[max_pt_x,min_pt_y],[min_pt_x,min_pt_y],[min_pt_x,max_pt_y]])
        cmin_pxl_L = np.zeros((pts.shape[0],pts.shape[1]))
        for pts_i,pts_d in enumerate(pts):
            cx0max,cx0min,cy0max,cy0min  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

            # max and min points in (x,y) coordinates
            cmin_pxl_L[pts_i,0] = (cx0min*pixel_width)
            cmin_pxl_L[pts_i,1] = (cy0min*pixel_width)

        x_L_pts.append(np.arange(cmin_pxl_L[3,0],cmin_pxl_L[0,0]+pixel_width,pixel_width))
        y_L_pts.append(np.arange(cmin_pxl_L[1,1],cmin_pxl_L[0,1]+pixel_width,pixel_width))

    x_L_lst.append(x_L_pts)
    y_L_lst.append(y_L_pts)
#ax.set_aspect(1)    

#plt.show()

## Define the outter mesh points
In this part, we use the x_L and y_L and composed them into a mesh grid that respects the grid

In [ ]:
from matplotlib.path import Path

xgrid = np.linspace(0,1,subpixel_grid)*pixel_width
ygrid = np.linspace(0,1,subpixel_grid)*pixel_width

chess_pts = []
chess_ptchs = []

xx = 5

centroid_L_arr = np.zeros((x0_L.shape[0],x0_L.shape[1],2))
for chess in range(x0_L.shape[0]):

    lst_pts = []
    ptchs_pts = []

    for point in range(x0_L.shape[1]):

        lst = []
        ptchs = []

        [X_crns,Y_crns] = np.meshgrid(x_L_lst[chess][point],y_L_lst[chess][point])

        for i in range(X_crns.flatten().shape[0]):
            cx0max = X_crns.flatten()[i]
            cy0max = Y_crns.flatten()[i]


            x_pixel = cx0max+xgrid
            y_pixel = cy0max+ygrid
            xx_pixel, yy_pixel = np.meshgrid(x_pixel,y_pixel)

            x, y = xx_pixel.flatten(), yy_pixel.flatten()
            points = np.vstack((x,y)).T 

            x_L = x_zhang_L[chess,point,:,:]*pixel_width
            x_L = tuple(x_L)
            p = Path(x_L) # make a polygon
            grid = p.contains_points(points)
            xxgrid = np.where(grid == True)

            lst.append((xxgrid[0].shape[0]/(subpixel_grid**2))*100)


            ptchs.append(Rectangle((cx0max,cy0max), pixel_width,pixel_width))

        ####### estimated centroid #####
        centroid_L_x = np.sum(np.multiply(X_crns.flatten(),lst))/(np.sum(lst)) + pixel_width/2
        centroid_L_y = np.sum(np.multiply(Y_crns.flatten(),lst))/(np.sum(lst)) + pixel_width/2

        centroid_L_arr[chess,point,:]  = (centroid_L_x,centroid_L_y)

        lst_pts.append(lst)
        ptchs_pts.append(ptchs)
    
    chess_pts.append(lst_pts)
    chess_ptchs.append(ptchs_pts)

 



In [ ]:
#fig = plt.figure()
#ax = fig.add_subplot()
#ax.plot(pixelGridXX,pixelGridYY,'r')
#ax.plot(pixelGridXX.T,pixelGridYY.T,'b')

centroid_L_org = np.zeros((x0_L.shape[0],x0_L.shape[1],2))

for chess_b_i in range(x0_L.shape[0]):
    for point in range(x0_L.shape[1]):
        #ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
        max_pt_x = np.max(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        max_pt_y = np.max(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        min_pt_x = np.min(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        min_pt_y = np.min(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        #ax.plot(X_crns,Y_crns, color = 'r',ls = "None", marker = "*")

        ####### original centroid ######
        centroid_L_x_orig = np.sum(x_zhang_L[chess_b_i,point,:,0]*pixel_width)/x_zhang_L[chess_b_i,point,:,0].shape[0]
        centroid_L_y_orig = np.sum(x_zhang_L[chess_b_i,point,:,1]*pixel_width)/x_zhang_L[chess_b_i,point,:,0].shape[0]

        #ax.plot(centroid_L_x_orig,centroid_L_y_orig,'g*')
        centroid_L_org[chess_b_i,point,:]  = (centroid_L_x_orig,centroid_L_y_orig)


    #p = PatchCollection(chess_ptchs[chess_b_i][point], cmap=cm.jet, alpha=1)
    #pixel_covered_percent_arr = (np.array(chess_pts[chess_b_i][point])).flatten()
    # define the color
    #p.set_array(pixel_covered_percent_arr)
    # set color max and min
    #p.set_clim([0, 100])
    #ax.add_collection(p)

#fig.colorbar(p)
        

#plt.show()

In [ ]:
centroid_L_arr

array([[[ 3.43582836,  0.07039655],
        [ 3.43583186,  0.44883364],
        [ 3.43582848,  0.82744091],
        ...,
        [ 1.63504903,  4.1460455 ],
        [ 1.63504736,  4.47493495],
        [ 1.63504873,  4.80388117]],

       [[ 3.4358334 ,  2.13498622],
        [ 3.43582842,  2.51338339],
        [ 3.43583033,  2.89191   ],
        ...,
        [ 1.63505131,  5.94031476],
        [ 1.63504965,  6.26933096],
        [ 1.63504916,  6.59838775]],

       [[10.317602  ,  0.07039928],
        [10.31759712,  0.44883434],
        [10.31760267,  0.82744149],
        ...,
        [ 7.61642932,  4.14604677],
        [ 7.61643111,  4.47493761],
        [ 7.61642803,  4.80388427]],

       [[10.31760108,  2.13498905],
        [10.31760038,  2.51337923],
        [10.31759845,  2.89191232],
        ...,
        [ 7.6164297 ,  5.94031823],
        [ 7.61643016,  6.26932838],
        [ 7.61643083,  6.59838815]]])

In [70]:
centroid_L_org

array([[[ 3.4359746 ,  0.07040162],
        [ 3.4359746 ,  0.44889772],
        [ 3.4359746 ,  0.82739383],
        ...,
        [ 1.63505745,  4.14606285],
        [ 1.63505745,  4.47503376],
        [ 1.63505745,  4.80400467]],

       [[ 3.4359746 ,  2.13492584],
        [ 3.4359746 ,  2.51342201],
        [ 3.4359746 ,  2.89191818],
        ...,
        [ 1.63505745,  5.94044971],
        [ 1.63505745,  6.26942015],
        [ 1.63505745,  6.59839153]],

       [[10.31772232,  0.07040162],
        [10.31772232,  0.44889772],
        [10.31772232,  0.82739383],
        ...,
        [ 7.61634636,  4.14606285],
        [ 7.61634636,  4.47503376],
        [ 7.61634636,  4.80400467]],

       [[10.31772232,  2.13492584],
        [10.31772232,  2.51342201],
        [10.31772232,  2.89191818],
        ...,
        [ 7.61634636,  5.94044971],
        [ 7.61634636,  6.26942015],
        [ 7.61634636,  6.59839153]]])

In [71]:
np.sum(centroid_L_org-centroid_L_arr)

0.03255505464653871

In [62]:
0.0001065479354880744

0.0001065479354880744